In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
  #accessing the html page and storing it into a BS object
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

  #creating a list to append values
raw_data=[]
  #targeting the table within the page
table = soup.find('table', class_='wikitable sortable')

  #iterating through each row in the table and appending the fields into the list
for word in table.find_all('td'):
    value = word.text.replace('\n','')
    raw_data.append(value)
  
   #reshaping the list to a to 288 rows x 3 columns. Then creating a dataframe on the reshaped list
df_postcode = pd.DataFrame(np.reshape(raw_data,(288,3)),columns=['Postal Code','Borough','Neighbourhood'])

    #deleting the rows, by index, of those records that contain a Borough value of Not assigned. 
df_postcode.drop((df_postcode[df_postcode['Borough']=='Not assigned'].index), inplace=True) 

In [3]:
combined=[] 
    #casting the post codes to a set() to remove duplicates, then casting back into a list to iterate using the for loop below
unique_postcodes = list(set(df_postcode['Postal Code']))

  #iterating through the dataframe to combine the neighbourhoods based on unique postcode. This also will check for any not assigned neighbourhoods and assign them the borough value
for i in range(len(unique_postcodes)):
    #returns a dataframe containing only the records of the current postcode, [i], in the list of unique_postcodes
      groupedby_postcode = df_postcode[df_postcode['Postal Code'] == unique_postcodes[i]]
  
    #checking for any "not assigned" neighbourhoods, and then hardcoding them to the borough name
      if groupedby_postcode['Neighbourhood'].any() == 'Not assigned':
        groupedby_postcode['Neighbourhood'] = groupedby_postcode['Borough']
    
    #setting delimited for the join function of the neighbourhoods
      temp_combine_neighbourhoods=', '
    #adding fields together into a new list:
      #first parameter = the current postcode string value
      #second parameter = the current borough value, by string
      #third parameter = the current neighbourhood values, joined by the delimiter above
      combined.append([unique_postcodes[i],list(set(groupedby_postcode['Borough']))[0],temp_combine_neighbourhoods.join(list(set(groupedby_postcode['Neighbourhood'])))])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
   #reshaping the list to a to 103 rows x 3 columns. Then creating a dataframe on the reshaped list
df_combined = pd.DataFrame(np.reshape(combined,(103,3)),columns=['Postal Code','Borough','Neighbourhood'])
print(np.shape(df_combined))
df_combined.head(12)

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M5S,Downtown Toronto,"Harbord, University of Toronto"
1,M9M,North York,"Emery, Humberlea"
2,M4P,Central Toronto,Davisville North
3,M8W,Etobicoke,"Alderwood, Long Branch"
4,M4M,East Toronto,Studio District
5,M4B,East York,"Woodbine Gardens, Parkview Hill"
6,M2N,North York,Willowdale South
7,M6G,Downtown Toronto,Christie
8,M5C,Downtown Toronto,St. James Town
9,M5T,Downtown Toronto,"Chinatown, Kensington Market, Grange Park"


Part 2 - This cell reads in the csv from the website and merges the current combined dataframe with the coordinates dataframe. Then display the first 12 records.

In [5]:
    #reading in the latitude and longitude coordinates from the csv file
df_lat_long=pd.read_csv("https://cocl.us/Geospatial_data")
    #merging the two dataframes together based on postal code
df_merged = pd.merge(df_combined, df_lat_long, on='Postal Code')
    #displaying a few lines of the merged table
df_merged.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
1,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
6,M2N,North York,Willowdale South,43.770120,-79.408493
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
9,M5T,Downtown Toronto,"Chinatown, Kensington Market, Grange Park",43.653206,-79.400049
